In [1]:
import pandas as pd
import numpy as np 
import seaborn as sn
from sklearn.metrics import cohen_kappa_score,f1_score,accuracy_score, precision_score, recall_score, classification_report, roc_auc_score, \
    hamming_loss
from itertools import compress
import os 
LABELS_LIST = ['car', 'gym', 'happy', 'night', 'relax',
    'running', 'sad', 'summer', 'work', 'workout']

In [2]:
def get_user_results(model_output, groundtruth,LABELS_LIST,selected_labels):  
    model_output_rounded = np.round(model_output)
    model_output_rounded = np.clip(model_output_rounded, 0, 1)
    
    # Create a dataframe where we keep all the evaluations, starting by prediction accuracy
    accuracies_perclass = sum(model_output_rounded == groundtruth) / len(groundtruth)
    results_df = pd.DataFrame(columns=LABELS_LIST)
    results_df.index.astype(str, copy=False)
    percentage_of_positives_perclass = sum(groundtruth) / len(groundtruth)
    results_df.loc[0] = percentage_of_positives_perclass
    results_df.loc[1] = accuracies_perclass
    results_df.index = ['Ratio of positive samples', 'Model accuracy']

    # Get true negative ratio
    true_negative_ratio_perclass = sum((model_output_rounded == groundtruth)
                                       * (groundtruth == 0)) / (len(groundtruth) - sum(groundtruth))
    results_df.loc[2] = true_negative_ratio_perclass
    
    # compute additional metrics (AUC,f1,recall,precision)
    precision_perlabel = precision_score(groundtruth, model_output_rounded, average=None)
    recall_perlabel = recall_score(groundtruth, model_output_rounded, average=None)
    f1_perlabel = f1_score(groundtruth, model_output_rounded, average=None)
    results_df = results_df.append(
        pd.DataFrame([recall_perlabel, precision_perlabel, f1_perlabel], columns=LABELS_LIST))
    results_df.index = ['Ratio of positive samples', 'Model accuracy', 
                        "True negative ratio", "Recall", "Precision", "f1-score"]
    results_df = results_df[selected_labels.index[selected_labels]] 
    results_df['average'] = results_df.mean(numeric_only=True, axis=1)
    return results_df

## user embeddings results

In [3]:
EXPERIMENT_PATh = "/srv/workspace/research/user_based_contexts_tagging/experiments_results/user_embeddings_active/2020-04-28_11-59-23/"
test_ground_truth = pd.read_csv("/srv/workspace/research/user_based_contexts_tagging/GroundTruth/test_active_clipped.csv")
test_groundtruth_from_model = np.loadtxt(EXPERIMENT_PATh + "test_ground_truth_classes.txt",delimiter=',')
user_ids = np.loadtxt(EXPERIMENT_PATh + "user_ids.txt",delimiter=',')
track_ids = np.loadtxt(EXPERIMENT_PATh + "tracks_ids.txt",delimiter=',')
test_output = np.loadtxt(EXPERIMENT_PATh + "predictions.out",delimiter=',')

In [4]:
our_ground_truth = test_ground_truth.copy()
our_ground_truth.song_id = track_ids
our_ground_truth.user_id = user_ids
our_ground_truth.iloc[:,2:] = test_groundtruth_from_model

In [5]:
our_predictions = test_ground_truth.copy()
our_predictions.song_id = track_ids
our_predictions.user_id = user_ids
our_predictions.iloc[:,2:] = test_output

In [7]:
all_users_results = pd.DataFrame(columns=['Ratio of positive samples', 'Model accuracy', 
                        "True negative ratio", "Recall", "Precision", "f1-score"])
for user in our_ground_truth.user_id.unique():
    user_truth = our_ground_truth[our_ground_truth.user_id == user]
    user_preds = our_predictions[our_predictions.user_id == user]
    active_labels = user_truth.sum() > 0
    active_labels = active_labels[2:]
    user_results = get_user_results(user_preds.values[:,2:],user_truth.values[:,2:],LABELS_LIST,active_labels)
    user_results = user_results["average"].T.values
    all_users_results.loc[user] = user_results

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
all_users_results.describe()

,Ratio of positive samples,Model accuracy,True negative ratio,Recall,Precision,f1-score
count,1399.000000,1399.000000,742.000000,1399.000000,1399.000000,1399.000000
mean,0.720785,0.614129,0.479885,0.661343,0.756574,0.661316
std,0.270351,0.163249,0.198824,0.161700,0.251557,0.180526
min,0.176471,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.500000,0.500000,0.325942,0.569722,0.520833,0.529186
50%,0.551724,0.600000,0.457738,0.675676,0.783654,0.655749
75%,1.000000,0.723544,0.633475,0.765924,1.000000,0.807692
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
summarized = all_users_results.describe()
summarized.to_csv(EXPERIMENT_PATh + "user_based_results_summary.csv")
all_users_results.to_csv(EXPERIMENT_PATh + "user_based_results.csv")

## Classic Model

In [10]:
EXPERIMENT_PATh = "/srv/workspace/research/user_based_contexts_tagging/experiments_results/classic_active/2020-04-28_12-29-52/"
test_ground_truth = pd.read_csv("/srv/workspace/research/user_based_contexts_tagging/GroundTruth/test_active_clipped.csv")
test_groundtruth_from_model = np.loadtxt(EXPERIMENT_PATh + "test_ground_truth_classes.txt",delimiter=',')
user_ids = np.loadtxt(EXPERIMENT_PATh + "user_ids.txt",delimiter=',')
track_ids = np.loadtxt(EXPERIMENT_PATh + "tracks_ids.txt",delimiter=',')
test_output = np.loadtxt(EXPERIMENT_PATh + "predictions.out",delimiter=',')

In [11]:
our_ground_truth = test_ground_truth.copy()
our_ground_truth.song_id = track_ids
our_ground_truth.user_id = user_ids
our_ground_truth.iloc[:,2:] = test_groundtruth_from_model

In [12]:
our_predictions = test_ground_truth.copy()
our_predictions.song_id = track_ids
our_predictions.user_id = user_ids
our_predictions.iloc[:,2:] = test_output

In [13]:
all_users_results = pd.DataFrame(columns=['Ratio of positive samples', 'Model accuracy', 
                        "True negative ratio", "Recall", "Precision", "f1-score"])
for user in our_ground_truth.user_id.unique():
    user_truth = our_ground_truth[our_ground_truth.user_id == user]
    user_preds = our_predictions[our_predictions.user_id == user]
    active_labels = user_truth.sum() > 0
    active_labels = active_labels[2:]
    user_results = get_user_results(user_preds.values[:,2:],user_truth.values[:,2:],LABELS_LIST,active_labels)
    user_results = user_results["average"].T.values
    all_users_results.loc[user] = user_results

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [14]:
summarized = all_users_results.describe()
summarized.to_csv(EXPERIMENT_PATh + "user_based_results_summary.csv")
all_users_results.to_csv(EXPERIMENT_PATh + "user_based_results.csv")

In [15]:
all_users_results.describe()

,Ratio of positive samples,Model accuracy,True negative ratio,Recall,Precision,f1-score
count,1399.000000,1399.000000,742.000000,1399.000000,1399.000000,1399.000000
mean,0.720785,0.645354,0.402104,0.732599,0.753268,0.688643
std,0.270351,0.201706,0.220756,0.195541,0.251331,0.200503
min,0.176471,0.029412,0.000000,0.029412,0.169405,0.057143
25%,0.500000,0.500000,0.232197,0.614949,0.520787,0.536118
50%,0.551724,0.617647,0.373857,0.750000,0.763986,0.670409
75%,1.000000,0.793103,0.527604,0.888889,1.000000,0.864932
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Deeper model

In [30]:
EXPERIMENT_PATh = "/srv/workspace/research/user_based_contexts_tagging/experiments_results/user_embeddings_deeper/2020-05-02_10-36-37/"
test_ground_truth = pd.read_csv("/srv/workspace/research/user_based_contexts_tagging/GroundTruth/test_active_clipped.csv")
user_ids = np.loadtxt(EXPERIMENT_PATh + "user_ids.txt",delimiter=',')
track_ids = np.loadtxt(EXPERIMENT_PATh + "tracks_ids.txt",delimiter=',')

EXPERIMENT_PATh = "/srv/workspace/research/user_based_contexts_tagging/experiments_results/user_embeddings_deeper/2020-04-21_14-55-56/"
test_groundtruth_from_model = np.loadtxt(EXPERIMENT_PATh + "test_ground_truth_classes.txt",delimiter=',')
test_output = np.loadtxt(EXPERIMENT_PATh + "predictions.out",delimiter=',')

In [31]:
our_ground_truth = test_ground_truth.copy()
our_ground_truth.song_id = track_ids
our_ground_truth.user_id = user_ids
our_ground_truth.iloc[:,2:] = test_groundtruth_from_model

In [32]:
our_predictions = test_ground_truth.copy()
our_predictions.song_id = track_ids
our_predictions.user_id = user_ids
our_predictions.iloc[:,2:] = test_output

In [33]:
all_users_results = pd.DataFrame(columns=['Ratio of positive samples', 'Model accuracy', 
                        "True negative ratio", "Recall", "Precision", "f1-score"])
for idx,user in enumerate(our_ground_truth.user_id.unique()):
    user_truth = our_ground_truth[our_ground_truth.user_id == user]
    user_preds = our_predictions[our_predictions.user_id == user]
    active_labels = user_truth.sum() > 0
    active_labels = active_labels[2:]
    user_results = get_user_results(user_preds.values[:,2:],user_truth.values[:,2:],LABELS_LIST,active_labels)
    user_results = user_results["average"].T.values
    all_users_results.loc[user] = user_results

0


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
all_users_results.describe()

,Ratio of positive samples,Model accuracy,True negative ratio,Recall,Precision,f1-score
count,1399.000000,1399.000000,1399.000000,1399.000000,1399.000000,1399.000000
mean,0.108349,0.371357,0.305543,0.852557,0.135864,0.225291
std,0.008634,0.024451,0.026542,0.071424,0.016630,0.024222
min,0.100000,0.275000,0.179302,0.600000,0.089765,0.152934
25%,0.101818,0.357143,0.292491,0.808333,0.124682,0.209040
50%,0.105714,0.372414,0.307703,0.857540,0.134006,0.223810
75%,0.111396,0.386207,0.321634,0.906508,0.145146,0.239652
max,0.152709,0.482759,0.408323,1.000000,0.207683,0.316071


In [35]:
summarized = all_users_results.describe()
summarized.to_csv(EXPERIMENT_PATh + "user_based_results_summary.csv")
all_users_results.to_csv(EXPERIMENT_PATh + "user_based_results.csv")

## Progressive

In [38]:
EXPERIMENT_PATh = "/srv/workspace/research/user_based_contexts_tagging/experiments_results/user_embeddings_deeper/2020-05-02_10-36-37/"
test_ground_truth = pd.read_csv("/srv/workspace/research/user_based_contexts_tagging/GroundTruth/test_active_clipped.csv")
user_ids = np.loadtxt(EXPERIMENT_PATh + "user_ids.txt",delimiter=',')
track_ids = np.loadtxt(EXPERIMENT_PATh + "tracks_ids.txt",delimiter=',')

EXPERIMENT_PATh = "/srv/workspace/research/user_based_contexts_tagging/experiments_results/user_embeddings_progressive/2020-05-03_11-56-57/"
test_groundtruth_from_model = np.loadtxt(EXPERIMENT_PATh + "test_ground_truth_classes.txt",delimiter=',')
test_output = np.loadtxt(EXPERIMENT_PATh + "predictions.out",delimiter=',')

In [39]:
our_ground_truth = test_ground_truth.copy()
our_ground_truth.song_id = track_ids
our_ground_truth.user_id = user_ids
our_ground_truth.iloc[:,2:] = test_groundtruth_from_model

In [40]:
our_predictions = test_ground_truth.copy()
our_predictions.song_id = track_ids
our_predictions.user_id = user_ids
our_predictions.iloc[:,2:] = test_output

In [41]:
all_users_results = pd.DataFrame(columns=['Ratio of positive samples', 'Model accuracy', 
                        "True negative ratio", "Recall", "Precision", "f1-score"])
for idx,user in enumerate(our_ground_truth.user_id.unique()):
    user_truth = our_ground_truth[our_ground_truth.user_id == user]
    user_preds = our_predictions[our_predictions.user_id == user]
    active_labels = user_truth.sum() > 0
    active_labels = active_labels[2:]
    user_results = get_user_results(user_preds.values[:,2:],user_truth.values[:,2:],LABELS_LIST,active_labels)
    user_results = user_results["average"].T.values
    all_users_results.loc[user] = user_results

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [42]:
all_users_results.describe()

,Ratio of positive samples,Model accuracy,True negative ratio,Recall,Precision,f1-score
count,1399.000000,1399.000000,1399.000000,1399.000000,1399.000000,1399.000000
mean,0.108349,0.108642,0.000362,0.999342,0.108332,0.190013
std,0.008634,0.008669,0.001094,0.006787,0.008620,0.013267
min,0.100000,0.096875,0.000000,0.857143,0.096976,0.162497
25%,0.101818,0.102273,0.000000,1.000000,0.101786,0.179523
50%,0.105714,0.106061,0.000000,1.000000,0.105769,0.185997
75%,0.111396,0.114035,0.000000,1.000000,0.111525,0.197034
max,0.152709,0.147783,0.008621,1.000000,0.147783,0.250891


In [35]:
summarized = all_users_results.describe()
summarized.to_csv(EXPERIMENT_PATh + "user_based_results_summary.csv")
all_users_results.to_csv(EXPERIMENT_PATh + "user_based_results.csv")